In [94]:
import json
import requests as re
import pandas as pd

In [95]:
def dados(tipo,id):
    response = re.get("https://pokeapi.co/api/v2/"+ tipo +"/" + (id))
    return response.json()

    
def dados_pokemon_all(grupo):
    response = re.get("https://pokeapi.co/api/v2/" + grupo +"/")
    print(response.status_code)
    return response.json()

def flatten_data(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

def select_colunas(df):
    df = pd.DataFrame(df)
    df = df.columns * (df.applymap(type) == str).all(0)
    df.replace(regex="",value=float("NaN"), inplace=True)
    df.dropna(axis="index", inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df.tolist()
    return df
    
def select_valores_dict(df,agg):
    df = pd.DataFrame(df)
    df = df.columns * (df.applymap(type) == int).all(0)
    df.replace(regex="",value=float("NaN"), inplace=True)
    df.dropna(axis="index", inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df.tolist()
    df_dict = {}
    for x in df:
        df_dict[x] = agg
    return df_dict

In [96]:
F_fato = pd.DataFrame(dados_pokemon_all("pokemon")["results"])
F_fato.columns = ["pokemon","ID_FATO"]

FD_pokemon= pd.DataFrame([])
for id in range(1, F_fato["pokemon"].count()+1):
    SUP_linha = dados("pokemon",str(id))
    SUP_pokemon_dimensao = pd.DataFrame(flatten_data(SUP_linha),index=flatten_data(SUP_linha).keys())
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "sprite")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "moves")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "game")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "is_hidden")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "is_default")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "slot")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "effort")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "forms")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "held")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "abilities_2")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "abilities_1")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(["id","order","name","species_name","species_url"], axis = 1, inplace = True)
    SUP_pokemon_dimensao["ID_FATO"]="https://pokeapi.co/api/v2/pokemon/"+ str(id) + "/"
    SUP_pokemon_dimensao = SUP_pokemon_dimensao.iloc[:1].reset_index(drop=True)
    FD_pokemon= pd.concat([FD_pokemon,SUP_pokemon_dimensao])
FD_pokemon.set_index("ID_FATO", inplace = True)

200


In [97]:
SUP_col =list(FD_pokemon.columns)

for x in range (0,6):
    SUP_col[SUP_col.index("stats_"+ str(x) +"_base_stat")] = FD_pokemon["stats_"+ str(x) +"_stat_name"][0]

SUP_col[SUP_col.index("abilities_0_ability_name")] = "ability_passive" 
SUP_col[SUP_col.index("types_0_type_name")] = "type" 
SUP_col[SUP_col.index("types_0_type_url")] = "type_url"
SUP_col[SUP_col.index("abilities_0_ability_url")] = "ability_url"
 
FD_pokemon.columns = SUP_col
FD_pokemon.drop(list(FD_pokemon.filter(regex = "stats")), axis = 1, inplace = True)


In [98]:
FD_ability= pd.DataFrame([])

for id in range(1, 150):
    SUP_linha = dados("ability",str(id))
    SUP_pokemon_dimensao = pd.DataFrame(flatten_data(SUP_linha),index=flatten_data(SUP_linha).keys())
    SUP_pokemon_dimensao = SUP_pokemon_dimensao.query('effect_entries_1_language_name=="en"')
    SUP_col = list(SUP_pokemon_dimensao.columns)
    SUP_col[SUP_col.index("effect_entries_1_short_effect")] = "ability_description"
    SUP_pokemon_dimensao.columns = SUP_col
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "name")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "pokemon")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "effect")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "flavor")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "generation_url")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "is_main_series")), axis = 1, inplace = True)
    SUP_pokemon_dimensao.drop(list(SUP_pokemon_dimensao.filter(regex = "id")), axis = 1, inplace = True)
    SUP_pokemon_dimensao["ability_url"]="https://pokeapi.co/api/v2/ability/"+ str(id) + "/"
    SUP_pokemon_dimensao = SUP_pokemon_dimensao.iloc[:1].reset_index(drop=True)
    FD_ability= pd.concat([FD_ability,SUP_pokemon_dimensao])



In [100]:
FA_pokemon = F_fato.merge(FD_pokemon, how='left', on="ID_FATO")
FA_pokemon = FA_pokemon.merge(FD_ability, how='left', on="ability_url")
FA_pokemon.drop(list(FA_pokemon.filter(regex = "type_")), axis = 1, inplace = True)
FA_pokemon.drop(list(FA_pokemon.filter(regex = "types")), axis = 1, inplace = True)
